In [146]:
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

In [147]:
# cur_task = "ProbabilisticReasoning"
# cur_task = "HorizonTask"
cur_task = "RestlessBandit"
# cur_task = "InstrumentalLearning"
# cur_task = "TwoStepTask"
# cur_task = "TemporalDiscounting"

# model = "Llama-3.1-8B-Instruct"
model = "Qwen3-0.6B"
# model = "Qwen3-32B"
# model = "gemma-2-9b-it"
str_strs = [0.0, 0.25]

In [148]:
res_dir = "Experiments/" + cur_task + "/"
score_csv = res_dir + "scores_data.csv"

In [149]:
results = {}


with open(score_csv, "r") as f:
    lines = f.readlines()
    columns = lines[0].strip().split(",")
    lines = lines[1:]
    lines = [line.strip().split(",") for line in lines]
    for line in lines:
        if "str_" not in line[0]:
            continue
        if model not in line[0]:
            continue
        steer_str, llm = line[0].split("+")
        if steer_str == "str_none":
            str_type = "none"
            str_str = 0.0
        else:
            str_type, str_str = steer_str.split("_")[1:]
            str_str = float(str_str)
        if str_type not in results:
            results[str_type] = {}
        results[str_type][str_str] = {}
        for i in range(1, len(columns)):
            if columns[i] not in results[str_type][str_str]:
                results[str_type][str_str][columns[i]] = []
            if len(line[i]) == 0:
                results[str_type][str_str][columns[i]].append('0.0')
            else:
                results[str_type][str_str][columns[i]].append(line[i])
        

In [150]:
print(results['random 0'])

{0.25: {'run': ['9'], 'performance_score1': ['51.38024390243903'], 'performance_score1_name': ['rewards mean'], 'behaviour_score1': ['0.6891682926829269'], 'behaviour_score1_name': ['meta-cognitive sensitivity'], 'behaviour_score2': ['0.99'], 'behaviour_score2_name': ['confidence mean'], 'performance_score2': ['0.6829268292682927'], 'performance_score2_name': ['mean accuracy'], 'behaviour_score3': ['0.6891682926829269'], 'behaviour_score3_name': ['adjusted qsr']}}


In [151]:
keys = list(results.keys())
for key in keys:
    for str_str in results[key]:
        for metric in results[key][str_str]:
            if metric == "run" or "_name" in metric:
                continue
            results[key][str_str][metric] = np.array(results[key][str_str][metric], dtype=float)
            results[key][str_str][metric] = np.mean(results[key][str_str][metric])

In [152]:
print(results['random 0'])

{0.25: {'run': ['9'], 'performance_score1': 51.38024390243903, 'performance_score1_name': ['rewards mean'], 'behaviour_score1': 0.6891682926829269, 'behaviour_score1_name': ['meta-cognitive sensitivity'], 'behaviour_score2': 0.99, 'behaviour_score2_name': ['confidence mean'], 'performance_score2': 0.6829268292682927, 'performance_score2_name': ['mean accuracy'], 'behaviour_score3': 0.6891682926829269, 'behaviour_score3_name': ['adjusted qsr']}}


In [ ]:
symptom_names = list(results.keys())
for i in range(10):
    symptom_names.remove("random "+str(i))
symptom_names.remove("none")

In [154]:
symptom_names

['depressed mood',
 'low self-esteem',
 'negativity bias',
 'guilt',
 'risk-aversion',
 'self-destruction',
 'manic mood',
 'grandiosity',
 'positivity bias',
 'lack of remorse',
 'risk-seeking',
 'hostility']

In [155]:
metrics_full = list(results[symptom_names[0]][str_strs[1]].keys())
metrics = []
for metric in metrics_full:
    if metric.endswith("_name"):
        metrics.append(metric[:-5])

In [156]:
n_symptoms = len(symptom_names)
cols = 3
rows = (n_symptoms + 2) // cols
if rows == 0:
    rows = 1

In [157]:
for metric in metrics:
    fig, axes = plt.subplots(rows, cols, figsize=(15, 3 * rows))
    axes = axes.flatten()
    
    # plt.title('Good Scores on SocialChemistry101')
    max_val = None
    min_val = None
    for idx, symptom in enumerate(symptom_names):
        vals = []
        for str_str in str_strs:
            if str_str == 0.0:
                vals.append(results['none'][str_str][metric])
            else:
                vals.append(results[symptom][str_str][metric])
        axes[idx].plot(str_strs, vals, marker='o', label=symptom, color='blue')

        cur_max = max(vals)
        cur_min = min(vals)

        if max_val is None:
            max_val = cur_max
        else:
            max_val = max(max_val, cur_max)
        if min_val is None:
            min_val = cur_min
        else:
            min_val = min(min_val, cur_min)

        # vals = []
        # for str_str in str_strs:
        #     if str_str == 0.0:
        #         vals.append(results['none'][str_str][metric])
        #     else:
        #         vals.append(results[symptom][-str_str][metric])
        # axes[idx].plot(str_strs, vals, marker='o', linestyle='--', label=symptom + " Negative", color='red')

        # cur_max = max(vals)
        # cur_min = min(vals)
        # max_val = max(max_val, cur_max)
        # min_val = min(min_val, cur_min)

        random_values = []
        for i in range(10):
            cur_vals = []
            for str_str in str_strs:
                if str_str == 0.0:
                    cur_vals.append(results['none'][str_str][metric])
                else:
                    cur_vals.append(results["random "+str(i)][str_str][metric])
            random_values.append(cur_vals)
        random_values = np.array(random_values)
        random_mean = np.mean(random_values, axis=0)
        lower_bound = np.mean(random_values, axis=0) - stats.sem(random_values, axis=0)
        sem = stats.sem(random_values)
        ci = stats.t.ppf(0.95, len(random_values) - 1) * sem
        lower_bound = random_mean - ci
        upper_bound = random_mean + ci
        axes[idx].plot(str_strs, random_mean, marker='o', linestyle='--', label='Random', color='orange')
        axes[idx].fill_between(str_strs, lower_bound, upper_bound, color='orange', alpha=0.3, label='Random ±95% CI')

        axes[idx].set_title(symptom, fontsize=12, fontweight='bold')
        axes[idx].set_xlabel('Intensity')
        axes[idx].set_ylabel('Score')

    for i in range(len(symptom_names)):
        lower_bound = min_val - abs(min_val) * 0.05
        upper_bound = max_val + abs(max_val) * 0.05
        axes[i].set_ylim(lower_bound, upper_bound)

    for i in range(n_symptoms, len(axes)):
        fig.delaxes(axes[i])

    fig.suptitle(metric + " on "+ cur_task, fontsize=20, fontweight='bold')
    plt.tight_layout()
    fig_dir = "figures/" + model + cur_task + "_"
    cur_metric_name = results["random 0"][str_strs[1]][metric + '_name'][0]
    if "/" in cur_metric_name:
        cur_metric_name = cur_metric_name.replace("/", "_")
    plt.savefig(fig_dir + cur_metric_name + ".png", dpi=600)
    plt.close()